# Core

> Core functionality that is used over the whole library.  

In [2]:
#| default_exp core

In [3]:
#| hide
from nbdev.showdoc import *

In [1]:
#| export  

# Support LangChain and lllama-index for OpenAI and LlamaCpp
from langchain.llms import LlamaCpp as LlamaCppLangChain, OpenAI as OpenAILangChain
from langchain.embeddings import LlamaCppEmbeddings, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llama_index.llms import LlamaCPP as LlamaCppLlamaIndex, OpenAI as OpenAILlamaIndex
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

import dataclasses
from enum import auto, Enum
from typing import List, Tuple, Any, Dict
from importlib_resources import files
import local_resources.prompts as prompts
import yaml 
import os

In [5]:
#| export

# LIBRARY-WIDE CONSTANTS AND ENUMS 

GGUF_LOOKUP_URL = {
    "llama-2-70b-chat": "https://huggingface.co/TheBloke/Llama-2-70B-chat-GGUF/resolve/main/llama-2-70b-chat.Q5_K_M.gguf",
    "llama-2-13b-chat": "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    "llama-2-7b-chat": "https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf",
    "mistral-7b-instruct": "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q5_K_M.gguf"
}

class MODEL_TYPE(Enum):
    LlamaCpp = auto()
    OpenAI = auto()

class AGENT_TYPE(Enum):
    LangChain = auto()
    LlamaIndex = auto()


In [6]:
#| export

@dataclasses.dataclass
class TaskInstruction:
    """A class that provides structure for task instructions (to be converted to LangChain PromptTemplates)."""

    # The name of this instruction template
    name: str

    # The description of this instruction template
    description: str

    # System prompts
    system: str

    # Instruction prompts
    instruction: str
    
    # The input variables that this instruction template requires
    input_variables: List[str] = dataclasses.field(default_factory=list)

    def generate_prompt_template(self) -> PromptTemplate:
        return PromptTemplate(
            input_variables=self.input_variables, 
            template=self.system + '\n' +self.instruction
        )

    def generate_llama2_prompt_template(self) -> PromptTemplate:
        return PromptTemplate(
            input_variables=self.input_variables, 
            template='''<s>[INST]
                <<SYS>>'''+self.system+'''<</SYS>>
                '''+self.instruction+'''
                [/INST]</s>'''
        )

    def generate_simple_instruction(self, input_map) -> str:
        pt = self.generate_prompt_template(self)
        return pt.format(input_map)

class TaskInstructionRegistry:
    """A class that stores and tracks PromptTemplates for use within a given function."""

    # The name of this instruction template
    registry: Dict[str, TaskInstruction] = {}

    def register_new_instruction_template(self, dict:Dict[str, str], override: bool = True):
        # check if all required fields are present
        assert 'name' in dict, "name is required"
        assert 'description' in dict, "description is required"
        assert 'system' in dict, "system is required"
        assert 'instruction' in dict, "instruction is required"
        assert 'input_variables' in dict, "input_variables is required"

        name = dict['name']
        description = dict['description']
        system = dict['system']
        instruction = dict['instruction']
        input_variables = dict['input_variables']

        self.register_instruction_template( TaskInstruction(name, description, system, instruction, input_variables), override=override)

    def deregister_instruction_template(self, name: str)->TaskInstruction:
        instruction_template = self.registry.pop(name)
        return instruction_template

    def register_instruction_template(self, template: TaskInstruction, override: bool = False):
        """Register a new conversation template."""
        if not override:
            assert template.name not in self.registry, f"{template.name} has been registered."
        self.registry[template.name] = template

    def get_instruction_template(self, name: str) -> TaskInstruction:
        """Get a conversation template."""
        if name not in self.registry:
            raise ValueError(f"{name} has not been registered.")
        return self.registry[name]

    def load_prompts_from_yaml(self, file_name: str):
        prompts_yaml = files(prompts).joinpath(file_name).read_text()
        prompts_dict = yaml.safe_load(prompts_yaml)
        for pname in prompts_dict:
            dct = prompts_dict[pname]
            dct['name'] = pname
            self.register_new_instruction_template(dct)

    def __str__(self) -> str:
        out = "Registered instruction templates:\n"
        for name in self.registry:
            out += '- %s'%(name) + '\n'
        return out

# A global registry for all instruction templates
#global instructions
#instructions = TaskInstructionRegistry()
#instructions.load_prompts_from_yaml('tiab_prompts.yaml')

In [7]:
#| export

# download GGUF files from HuggingFace URL and save it to disk in defined directory, return local file path
def get_cached_gguf(gguf_file: str) -> str:
    import requests
    import os
    from tqdm import tqdm
    from pathlib import Path

    if gguf_file not in GGUF_LOOKUP_URL:
        raise ValueError(f"{gguf_file} has not been registered.")
    
    url = GGUF_LOOKUP_URL[gguf_file]
    local_dir = os.environ['LLMS_TEMP_DIR']

    # create local directory if not exists
    Path(local_dir).mkdir(parents=True, exist_ok=True)

    # download file
    local_filename = url.split('/')[-1]
    local_filepath = os.path.join(local_dir, local_filename)
    if not os.path.exists(local_filepath):
        r = requests.get(url, stream=True)
        with open(local_filepath, 'wb') as f:
            file_size = int(r.headers['content-length'])
            chunk_size = 1000
            with tqdm(ncols=100, desc="Downloading", total=file_size, unit_scale=True) as pbar:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)
    
    return local_filepath

def get_langchain_embeddings(llm_name, **kwargs):

    model_type = None
    if llm_name == 'gpt-4' :
        model_type = MODEL_TYPE.OpenAI
    elif GGUF_LOOKUP_URL.get(llm_name) is not None:
        model_type = MODEL_TYPE.LlamaCpp
    else:
        raise ValueError(f"Unknown LLM {llm_name}")

    if model_type == MODEL_TYPE.LlamaCpp:

        n_gpu_layers = kwargs.get('n_gpu_layers', 1)
        n_batch = kwargs.get('n_batch', 512)  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        model_path = get_cached_gguf(llm_name)
        n_ctx = kwargs.get('n_ctx', 4096)

        llme = LlamaCppEmbeddings(
            model_path=model_path,
            n_ctx=n_ctx,
            n_gpu_layers=n_gpu_layers,
            n_batch=n_batch,
            f16_kv=True,
            verbose=True, # Verbose is required to pass to the callback manager
        )        

        return llme
    
    elif model_type == MODEL_TYPE.OpenAI:
        
        if os.environ.get('OPENAI_API_KEY') is None:
            raise ValueError(f"OPENAI_API_KEY env. variable not set.")

        openai_api_key = os.environ['OPENAI_API_KEY']
        llme = OpenAIEmbeddings(openai_api_key=openai_api_key)
        
        return llme

    else:

        raise ValueError(f"Unknown model {model_type}")



def get_langchain_llm(llm_name, **kwargs):

    model_type = None
    if llm_name == 'gpt-4' :
        model_type = MODEL_TYPE.OpenAI
    elif GGUF_LOOKUP_URL.get(llm_name) is not None:
        model_type = MODEL_TYPE.LlamaCpp
    else:
        raise ValueError(f"Unknown LLM {llm_name}")

    if model_type == MODEL_TYPE.LlamaCpp:

        callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        n_gpu_layers = kwargs.get('n_gpu_layers', 1)
        temperature = kwargs.get('temperature', 0.1)
        n_batch = kwargs.get('n_batch', 512)  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        model_path = get_cached_gguf(llm_name)
        n_ctx = kwargs.get('n_ctx', 4096)

        llm = LlamaCppLangChain(
            model_path=model_path,
            n_ctx=n_ctx,
            n_gpu_layers=n_gpu_layers,
            temperature=temperature,
            n_batch=n_batch,
            callback_manager=callback_manager,
            f16_kv=True,
            verbose=True, # Verbose is required to pass to the callback manager
        )        

        return llm

    elif model_type == MODEL_TYPE.OpenAI:
        
        if os.environ.get('OPENAI_API_KEY') is None:
            raise ValueError(f"OPENAI_API_KEY env. variable not set.")

        openai_api_key = os.environ['OPENAI_API_KEY']
        llm = OpenAILangChain(openai_api_key=openai_api_key)
        
        return llm 

    else:

        raise ValueError(f"Unknown model {model_type}")
    
def get_llamaindex_llm(llm_name, **kwargs):

    model_type = None
    if llm_name == 'gpt-4':
        model_type = MODEL_TYPE.OpenAI
    elif GGUF_LOOKUP_URL.get(llm_name) is not None:
        model_type = MODEL_TYPE.LlamaCpp

    temperature = kwargs.get('temperature', 0.1)
    n_batch = kwargs.get('n_batch', 512)  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

    if model_type == MODEL_TYPE.LlamaCpp:

        model_path = get_cached_gguf(llm_name)
        n_ctx = kwargs.get('n_ctx', 4096)

        llm = LlamaCppLlamaIndex( 
            model_path=model_path,
            temperature=temperature,
            max_new_tokens=n_batch,
            context_window=n_ctx,
            generate_kwargs={},
            model_kwargs={"n_gpu_layers": 1},
            messages_to_prompt=messages_to_prompt,
            completion_to_prompt=completion_to_prompt,
            verbose=True)

        return llm

    elif model_type == MODEL_TYPE.OpenAI:
        
        if os.environ.get('OPENAI_API_KEY') is None:
            raise ValueError(f"OPENAI_API_KEY env. variable not set.")

        openai_api_key = os.environ['OPENAI_API_KEY']
        llm = OpenAILlamaIndex(model="gpt-4", temperature=temperature, max_tokens=n_batch)
        
        return llm 

    else:

        raise ValueError(f"Unknown model {model_type}")
